<a href="https://colab.research.google.com/github/shreyasseshadri/SC-Project/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [10]:
# # Load the Drive helper and mount
from google.colab import drive

# # This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras

In [0]:
import numpy as np
import random as rn
rn.seed(123)
np.random.seed(123)
from keras.models import Model
from keras.layers import Input, Dense,Conv2D ,LSTM,Lambda,Flatten,Dropout
from keras.utils import np_utils
from keras import backend as K
tf_session = K.get_session()
from keras import regularizers
import tensorflow as tf
tf.set_random_seed(123)

In [0]:
def stack_dim(X):
    conv_outputs=tf.unstack(X,axis=3)
    return tf.concat(conv_outputs,axis=1)

def get_shape(input_shape):
    assert len(input_shape)==4
    return (input_shape[0],input_shape[1]*input_shape[3],input_shape[2])

def model(input_shape,hidden_size,dense_layer1,dense_layer2):
    X_input=Input(input_shape)
    X=Conv2D(32, (5, 5), input_shape=input_shape, activation='relu')(X_input)
    X=Dropout(0.5)(X)
    X=Lambda(stack_dim,output_shape=get_shape)(X)
    X=LSTM(hidden_size,return_sequences=True)(X)
    X=Dense(dense_layer1, activation='relu',kernel_regularizer=regularizers.l2(0.01))(X)
    X=Flatten()(X)
    X=Dropout(0.5)(X)
    X=Dense(dense_layer2, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='sent_classifier')
    return model

In [0]:
X_train=np.load('drive/My Drive/SC-Project/train_data.npy')

In [0]:
X_test=np.load('drive/My Drive/SC-Project/X_test.npy')

In [0]:
y_train=np.load('drive/My Drive/SC-Project/y_train.npy')
y_test=np.load('drive/My Drive/SC-Project/y_test.npy')

In [0]:
m=model(X_train.shape[1:],128,50,2)

In [0]:
from keras.optimizers import SGD,Adam
# opt=SGD(lr=0.05, momentum=0.01,decay=0.0, nesterov=False)
m.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
def train(l,h):
    train_size=h-l
    train_pos=np.where(y_train[l:h][...,1]==1)[0]+l
    train_neg=np.where(y_train[l:h][...,0]==1)[0]+l
    np.random.seed(123)
    pos=np.random.choice(train_pos,train_size//2)
    np.random.seed(123)
    neg=np.random.choice(train_neg,train_size-train_size//2)
    indices=np.append(pos,neg,0)
    np.random.seed(123)
    np.random.shuffle(indices)
    return indices

In [0]:
train_index=train(0,2210)
val_index=train(4000,8000)

In [15]:
val_index

array([4323, 4685, 6487, ..., 7127, 7990, 7329])

In [0]:
from sklearn.utils import resample
ups_train_index = resample(train_index,replace=True,n_samples=4000,random_state=123)

In [18]:
len(ups_train_index)

4000

In [0]:
hist=m.fit(X_train[ups_train_index],y_train[ups_train_index],batch_size=64,epochs=20,verbose=1,validation_data=(X_train[val_index],y_train[val_index]))

In [0]:
m.save('drive/My Drive/SC-Project/final.h5')

In [0]:
from keras.models import load_model
m = load_model('drive/My Drive/SC-Project/final.h5',custom_objects={"tf": tf})

In [20]:
pred=m.evaluate(X_test,y_test)
print(pred)

25000/25000 [==============================] - 4102s 164ms/step
[0.7009602817535401, 0.90712]
